### 变量的作用域

In [2]:
b = 6
def f3(a):
    global b #全局变量，函数内部定义的变量默认为局部（local）变量
    print(a)
    print(b)
    b = 9 #全局变量

In [3]:
f3(3)

3
6


In [4]:
from dis import dis #反编译的使用

In [5]:
dis(f3)

  4           0 LOAD_GLOBAL              0 (print)
              3 LOAD_FAST                0 (a)
              6 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
              9 POP_TOP

  5          10 LOAD_GLOBAL              0 (print)
             13 LOAD_GLOBAL              1 (b)
             16 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
             19 POP_TOP

  6          20 LOAD_CONST               1 (9)
             23 STORE_GLOBAL             1 (b)
             26 LOAD_CONST               0 (None)
             29 RETURN_VALUE


### 比包
> 闭包指延伸了作用域的函数

In [7]:
# Aver_v1
class Averager():
    
    """
    计算平均数，保留原录入数据
    """
    def __init__(self):
        self.series = []
    def __call__(self,new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

In [8]:
avg = Averager()

In [9]:
avg(3)

3.0

In [10]:
avg(8)

5.5

In [11]:
#Aver_v2
#高阶函数实现计算平均数
def make_averager():
    series = []
    def averager(new_value):
        series.append(new_value) #series 是自由变量，指未在本地绑定的变量
        total = sum(series)
        return total/len(series)
    return averager

In [12]:
avg = make_averager()

In [13]:
avg(10)

10.0

In [14]:
avg(20)

15.0

In [15]:
avg.__code__.co_varnames

('new_value', 'total')

In [17]:
avg.__code__.co_freevars

('series',)

In [18]:
avg.__closure__

(<cell at 0x03EBFE90: list object at 0x03EC9B48>,)

In [19]:
avg.__closure__[0].cell_contents

[10, 20]

In [20]:
#Aver_v3
def make_averager_v3():
    count = 0
    total = 0
    def averager(new_value):
        count += 1 # 因是不可变变量，重新赋值会变成局部变量（隐含创建新的变量）
        total += new_value
        return total/count
    return averager

In [21]:
avg = make_averager_v3()

In [22]:
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [23]:
#Aver_v4
def make_averager_v4():
    count = 0
    total = 0
    def averager(new_value):
        nonlocal count,total
        count += 1 # 因是不可变变量，重新赋值会变成局部变量（隐含创建新的变量）
        total += new_value
        return total/count
    return averager

In [24]:
avg = make_averager_v4()

In [25]:
avg(10)

10.0

In [26]:
avg(20)

15.0

In [30]:
import time
def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        msg = '[%0.8fs] %s(%s) ->%r' %(elapsed,name,arg_str,result)
        print(msg)
        return result
    return clocked

### 装饰器
> 典型行为是，把被装饰的函数替换成新函数，二者接受相同的参数，通常返回被装饰函数本该返回的值，同时还做些其他一些事情。

In [31]:
@clock
def snooze(seconds): # 实际等于snooze = clocked(fanc),func成为一自由变量，内部函数可访问
    time.sleep(seconds)

In [32]:
snooze(3)

[2.99927327s] snooze(3) ->None


In [33]:
@clock
def factorial(n): #factorial = clock(factorial)
    return 1 if n <2 else n * factorial(n -1)

In [35]:
factorial(5)

[0.00000276s] factorial(1) ->1
[0.00166150s] factorial(2) ->2
[0.00180322s] factorial(3) ->6
[0.00187506s] factorial(4) ->24
[0.00193941s] factorial(5) ->120


120

In [1]:
%%writefile clockdeco.py
#内省中能使用原函数名称，functools模块
import time
import functools
def clock(func):
    @functools.wraps(func)
    def clocked(*args,**kwargs):
        t0 = time.perf_counter()
        result = func(*args,**kwargs)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs: #可以使用关键字参数
            pairs = ['%s=%r' % (k,w) for k,w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        msg = '[%0.8fs] %s(%s) ->%r' %(elapsed,name,arg_str,result)
        print(msg)
        return result
    return clocked

Overwriting clockdeco.py


### 标准库中的装饰器

* lru_cache()
* singledispatch

In [2]:
from clockdeco import clock

In [3]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)
if __name__ == '__main__':
    print(fibonacci(6))

[0.00000237s] fibonacci(0) ->0
[0.00000237s] fibonacci(1) ->1
[0.00034817s] fibonacci(2) ->1
[0.00000158s] fibonacci(1) ->1
[0.00000158s] fibonacci(0) ->0
[0.00000118s] fibonacci(1) ->1
[0.00014211s] fibonacci(2) ->1
[0.00028264s] fibonacci(3) ->2
[0.00077094s] fibonacci(4) ->3
[0.00000118s] fibonacci(1) ->1
[0.00000118s] fibonacci(0) ->0
[0.00000118s] fibonacci(1) ->1
[0.00013935s] fibonacci(2) ->1
[0.00028303s] fibonacci(3) ->2
[0.00000158s] fibonacci(0) ->0
[0.00000158s] fibonacci(1) ->1
[0.00020408s] fibonacci(2) ->1
[0.00000237s] fibonacci(1) ->1
[0.00000158s] fibonacci(0) ->0
[0.00000118s] fibonacci(1) ->1
[0.00013816s] fibonacci(2) ->1
[0.00028106s] fibonacci(3) ->2
[0.00063160s] fibonacci(4) ->3
[0.00105082s] fibonacci(5) ->5
[0.00196545s] fibonacci(6) ->8
8


In [6]:
import functools
from clockdeco import clock
@functools.lru_cache()
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2)+fibonacci(n-2)
if __name__ == '__main__':
    print(fibonacci(15))

[0.00000118s] fibonacci(1) ->1
[0.00022698s] fibonacci(3) ->2
[0.00027514s] fibonacci(5) ->4
[0.00031027s] fibonacci(7) ->8
[0.00034343s] fibonacci(9) ->16
[0.00037580s] fibonacci(11) ->32
[0.00040896s] fibonacci(13) ->64
[0.00048238s] fibonacci(15) ->128
128


In [7]:
from functools import singledispatch
from collections import abc
import numbers
import html

In [9]:
@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)
@htmlize.register(str)
def _(text):
    content = html.escape(text).replace('\n','<br>\n')
    return '<p>{0}</p>'.format(content)
@htmlize.register(numbers.Integral)
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)
@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>' .join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

In [10]:

%%writefile registration.py
registry = []
def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

Writing registration.py


In [11]:
from registration import register

In [12]:
@register
def f1():
    print('running f1()')

running register(<function f1 at 0x03F5CDF8>)


In [13]:
f1()

running f1()
